In [1]:
# This Notebook is focused on cleaning data
print("What hath God wrought")

What hath God wrought


In [2]:
import numpy as np
import pandas as pd

In [3]:
# Read in csv files as dataframes
airlines_df = pd.read_csv("data/airlines.csv")
airports_df = pd.read_csv("data/airports.csv")
# flights_df is the main DataFrame with
flights_df = pd.read_csv("data/flights.csv",
                         dtype={"YEAR":"category", "MONTH":"uint8", "DAY":"uint8", "DAY_OF_WEEK":"uint8", "AIRLINE":"category", "FLIGHT_NUMBER":"category", "TAIL_NUMBER":"category", "ORIGIN_AIRPORT":"str", "DESTINATION_AIRPORT":"str", "SCHEDULED_DEPARTURE":"uint16", "SCHEDULED_ARRIVAL":"uint16", "DIVERTED":"int8", "CANCELLED":"int8"})
print(f"flights_df is the main dataframe with flights_df.shape[0] rows.  Each row is a flight")

flights_df is the main dataframe with flights_df.shape[0] rows.  Each row is a flight


In [4]:
flights_df.head()
# flights_df has 31 columns, columns for YEAR, MONTH, DAY, DAY_OF_WEEK, and SCHEDULED DEPARTURE (

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# The major issue with the dataset is that the Airport IATA codes (3 character codes such as JFK or LAX) are replaced by 5 digit numbers for the month of October.
# These numbers can be found at the address below:
# https://transtats.bts.gov/FieldInfo.asp?Svryq_Qr5p=b4vtv0%FDNv42146%FP%FDNv42146%FDVQ.%FDN0%FDvqr06vsvpn6v10%FD07zor4%FDn55vt0rq%FDoB%FDhf%FDQbg%FD61%FDvqr06vsB%FDn%FD70v37r%FDnv42146.%FD%FDh5r%FD6uv5%FDsvryq%FDs14%FDnv42146%FDn0nyB5v5%FDnp4155%FDn%FD4n0tr%FD1s%FDBrn45%FDorpn75r%FDn0%FDnv42146%FDpn0%FDpun0tr%FDv65%FDnv42146%FDp1qr%FDn0q%FDnv42146%FDp1qr5%FDpn0%FDor%FD4r75rq.&Svryq_gB2r=a7z&Y11x72_gnoyr=Y_NVecbeg_VQ&gnoyr_VQ=FIL&flf_gnoyr_anzr=g_gEDDQ_ZNeXRg_hf_PNeeVRe_baYl&fB5_Svryq_anzr=beVTVa_NVecbeg_VQ
numeric_flights_df = flights_df[flights_df["ORIGIN_AIRPORT"].str.contains(r"\d+", regex=True)]
print("Number of flights in October:", len(flights_df[flights_df["MONTH"] == 10]))
print("Number of October flights with a numeric code:", numeric_flights_df['MONTH'].value_counts().loc[10])

Number of flights in October: 486165
Number of October flights with a numeric code: 486165


In [6]:
# I will replace the 5 digit codes with the 3 character IATA codes by matching the data together.  In order to this I will have to clean the data and extract derivative columns.

codes = pd.concat([numeric_flights_df["ORIGIN_AIRPORT"], numeric_flights_df["DESTINATION_AIRPORT"]], ignore_index=True).unique().astype("int64") # List of all the IATA codes in the dataset

numeric_airports_df = pd.read_csv("data/L_AIRPORT_ID.csv").set_index("Code") # List of all airports and their IATA codes as well as the 5 digit codes
numeric_airports_df = numeric_airports_df.loc[codes].reset_index() # Only including the codes of airports that are in the flights_df dataset in October
numeric_airports_df["Code"] = numeric_airports_df["Code"].astype("str")
numeric_airports_df["N_CITY"] = numeric_airports_df["Description"].str.split(",").str[0] # Derivative column for the City
numeric_airports_df["AIRPORT"] = numeric_airports_df["Description"].str.split(": ").str[1] # Derivative column for the Airport name
numeric_airports_df["N_STATE"] = numeric_airports_df["Description"].str.extract(r", (.+?):") # Derivative column for the State

for c in ["N_CITY", "AIRPORT", "N_STATE"]:
    numeric_airports_df[c].str.strip()

print("Number of airports in the month of October:", len(numeric_airports_df))
print("Number of airports in the dataset as a whole:", len(airports_df))
# There are 15 less airports used in October than in the dataset as a whole
numeric_airports_df.head()

Number of airports in the month of October: 307
Number of airports in the dataset as a whole: 322


,Code,Description,N_CITY,AIRPORT,N_STATE
0,14747,"Seattle, WA: Seattle/Tacoma International",Seattle,Seattle/Tacoma International,WA
1,14771,"San Francisco, CA: San Francisco International",San Francisco,San Francisco International,CA
2,12889,"Las Vegas, NV: Harry Reid International",Las Vegas,Harry Reid International,NV
3,12892,"Los Angeles, CA: Los Angeles International",Los Angeles,Los Angeles International,CA
4,14869,"Salt Lake City, UT: Salt Lake City International",Salt Lake City,Salt Lake City International,UT


In [7]:
# Cleaning airports_df
airports_df["AIRPORT"] = airports_df["AIRPORT"].str.replace("\u00A0", "") # Get rid of all non-breaking spaces
airports_df["AIRPORT"] = airports_df["AIRPORT"].str.replace("Airport", "").str.strip() # Remove the word "Airport" to match with numeric_airports_df because the word "airport" is not commonly included in numeric_airports_df

In [8]:
# Match airports based off of their names
numeric_airports_df = pd.merge(numeric_airports_df, airports_df, on="AIRPORT", how="left")
matched_airports_df = numeric_airports_df.dropna()[["Code", "IATA_CODE"]]

# Store the unmatched airports, both their numeric codes, and their IATA codes.
unmatched_numeric_airports_df = numeric_airports_df.loc[numeric_airports_df["LATITUDE"].isna(), ["Code", "Description", "N_CITY", "AIRPORT", "N_STATE"]]
unmatched_airports_df = airports_df[~airports_df["IATA_CODE"].isin(matched_airports_df["IATA_CODE"].to_list())]


# 181 airports were successfully matched using the airport name
print("Number of airports successfully matched:", len(matched_airports_df))
print("Number of unmatched airports from the month of October:", len(unmatched_numeric_airports_df))
print("Number of unmatched airports from the dataset as a whole:", len(unmatched_airports_df))

Number of airports successfully matched: 181
Number of unmatched airports from the month of October: 126
Number of unmatched airports from the dataset as a whole: 141


In [9]:
# Match airports by what city they are in

# Gather how many times each city occurs
unmatched_numeric_cities = unmatched_numeric_airports_df["N_CITY"].value_counts()
unmatched_cities = unmatched_airports_df["CITY"].value_counts()

# Find when the city occurs only once in both the numeric and IATA DataFrames.
by_city_airports_df =  pd.merge(unmatched_numeric_airports_df.set_index("N_CITY")
                                .loc[unmatched_numeric_cities[unmatched_numeric_cities ==1].index],
                                unmatched_airports_df.set_index("CITY").loc[unmatched_cities[unmatched_cities == 1].index],
                                left_index=True, right_index=True, how="left")

# Update the matched_airports_df
matched_by_city_airports_df = by_city_airports_df[["Code", "IATA_CODE"]].dropna()
matched_airports_df = pd.concat([matched_airports_df, matched_by_city_airports_df])
print("Number of airports successfully matched:", len(matched_airports_df))

# Update both datasets of unmatched airport codes
unmatched_numeric_airports_df = unmatched_numeric_airports_df[~unmatched_numeric_airports_df["Code"].isin(matched_by_city_airports_df["Code"].to_list())]
unmatched_airports_df = unmatched_airports_df[~unmatched_airports_df["IATA_CODE"].isin(matched_by_city_airports_df["IATA_CODE"].to_list())]
print("Number of unmatched airports from the month of October:", len(unmatched_numeric_airports_df))
print("Number of unmatched airports from the dataset as a whole:", len(unmatched_airports_df))

Number of airports successfully matched: 278
Number of unmatched airports from the month of October: 29
Number of unmatched airports from the dataset as a whole: 44


In [10]:
# Match airports by what state they are in

# Gather how many times each State occurs
unmatched_numeric_states = unmatched_numeric_airports_df["N_STATE"].value_counts()
unmatched_states = unmatched_airports_df["STATE"].value_counts()

# Find when the State occurs only once in both the numeric and IATA DataFrames.
by_state_airports_df = pd.merge(unmatched_numeric_airports_df.set_index("N_STATE")
                                .loc[unmatched_numeric_states[unmatched_numeric_states ==1].index],
                                unmatched_airports_df.set_index("STATE").loc[unmatched_states[unmatched_states == 1].index],
                                left_index=True, right_index=True, how="left")

# Update the matched_airports_df
matched_by_state_airports_df = by_state_airports_df[["Code", "IATA_CODE"]].dropna()
matched_airports_df = pd.concat([matched_airports_df, matched_by_state_airports_df])
print("Number of airports successfully matched:", len(matched_airports_df))

# Update both datasets of unmatched airport codes
unmatched_airports_df = unmatched_airports_df[~unmatched_airports_df["IATA_CODE"].isin(matched_by_state_airports_df["IATA_CODE"].to_list())]
unmatched_numeric_airports_df = unmatched_numeric_airports_df[~unmatched_numeric_airports_df["Code"].isin(matched_by_state_airports_df["Code"].to_list())]
print("Number of unmatched airports from the month of October:", len(unmatched_numeric_airports_df))
print("Number of unmatched airports from the dataset as a whole:", len(unmatched_airports_df))

Number of airports successfully matched: 291
Number of unmatched airports from the month of October: 16
Number of unmatched airports from the dataset as a whole: 31


In [11]:
# There are only 16 values left to match, so I will manually match them.
codes = {
    216: "SAN", 297: "MVY", 153: "EWN", 154: "OAJ", 177: "FAY", 92: "JFK", 107: "LGA", 265: "SWF", 130: "RDM", 296: "OTH",  17: "IAH", 26: "HOU", 49: "CLL", 52: "MAF",
    64: "MFE", 241: "SGU"
}
print(len(codes.keys()))
# Add the codes to the unmatched data and then add the to the matched data to have a complete dataset
unmatched_numeric_airports_df["IATA_CODE"] = ""
unmatched_numeric_airports_df.loc[codes.keys(), "IATA_CODE"] = list(codes.values())
matched_airports_df = pd.concat([matched_airports_df, unmatched_numeric_airports_df[["Code", "IATA_CODE"]]])

16


In [12]:
# Edit flights_df to replace the 5 digit codes with the IATA airport codes

for c in ["ORIGIN", "DESTINATION"]:
    matched_airports_df.columns=[f"{c}_AIRPORT", f"{c}_IATA_CODE"]
    # Merge the dataframes to add a column with the correct IATA code.
    flights_df = pd.merge(flights_df, matched_airports_df, on=f"{c}_AIRPORT", how="left")
    # Replace the 5 digit codes with the IATA codes
    flights_df.loc[flights_df["MONTH"] == 10, f"{c}_AIRPORT"] = flights_df[f"{c}_IATA_CODE"]
    # Drop the added column
    flights_df = flights_df.drop(f"{c}_IATA_CODE", axis=1)

# Check to see if there were any unedited columns
flights_df["ORIGIN_AIRPORT"].str.contains(r"\d+", regex=True).sum()

0

In [13]:
flights_df.head()
# flights_df has 31 columns

# For data that is recorded in minutes such as TAXI_IN taking x amount of minutes, I will leave those columns as is:
# DEPARTURE_DELAY, TAXI_OUT, SCHEDULED_TIME, ELAPSED_TIME, AIR_TIME, TAXI_IN, ARRIVAL_DELAY

# For data that is recorded in time and formated as hhmm such as WHEELS_ON happening at 08:21 but being recorded as 821, I will turn these columns into pd.datetime, but there are several problems:

# columns for YEAR, MONTH, DAY, DAY_OF_WEEK, are accurate for SCHEDULED_DEPARTURE rather than for the other various recorded times DEPARTURE_TIME, WHEELS_OFF, WHEELS_ON, SCHEDULED_ARRIVAL, ARRIVAL_TIME

# We can look at the first row in which the SCHEDULED_DEPARTURE is 00:05 on the 1st of January and the DEPARTURE_TIME is 23:54
# In order to find an accurate datetime for all 6 time columns, I will use the derivative columns in the dataset such as DEPARTURE_DELAY to calculate them.

# The columns related to the destination airport, WHEELS_ON, SCHEDULED_ARRIVAL, ARRIVAL TIME, are in the timezone of the destination airport rather than the origin airport.
# I will include in the correct destination timezone, the hour and minute of the SCHEDULED_ARRIVAL, but all other time will be in the timezone of the origin airport

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Converting SCHEDULED_DEPARTURE from hhmm into datetime and adding separate HOUR and MINUTE columns for departure
flights_df["SCHEDULED_DEPARTURE"] = flights_df["SCHEDULED_DEPARTURE"].astype(str).str.zfill(4) # Increasing the total number of digits to 4
flights_df["HOURS"] = flights_df["SCHEDULED_DEPARTURE"].str[:2].astype("uint8") # Creating the HOURS column
flights_df["MINUTES"] = flights_df["SCHEDULED_DEPARTURE"].str[2:].astype("uint8") # Creating the MINUTES column
flights_df["SCHEDULED_DEPARTURE"] = pd.to_datetime(flights_df[["YEAR", "MONTH", "DAY", "HOURS", "MINUTES"]]) # Transforming SCHEDULED_DEPARTURE into datetime
flights_df = flights_df.rename({"HOURS":"SCHEDULED_DEPARTURE_HOURS",
                                "MINUTES":"SCHEDULED_DEPARTURE_MINUTES"}, axis=1)

In [15]:
flights_df["SCHEDULED_ARRIVAL"] = flights_df["SCHEDULED_ARRIVAL"].astype(str).str.zfill(4) # Increasing the total number of digits to 4
flights_df["SCHEDULED_ARRIVAL_HOUR_IN_DESTINATION_TIMEZONE"] = flights_df["SCHEDULED_ARRIVAL"].str[:2].astype("uint8") # Creating the HOURS column
flights_df["SCHEDULED_ARRIVAL_MINUTE_IN_DESTINATION_TIMEZONE"] = flights_df["SCHEDULED_ARRIVAL"].str[2:].astype("uint8") # Creating the MINUTES column
flights_df["SCHEDULED_ARRIVAL"] = flights_df["SCHEDULED_DEPARTURE"] + pd.to_timedelta(flights_df["SCHEDULED_TIME"], unit="m") # Setting the SCHEDULED_ARRIVAL column into datetime in the timezone of the origin airport


In [16]:
# Creating datetime columns for the other time columns
flights_df["DEPARTURE_TIME"] = flights_df["SCHEDULED_DEPARTURE"] + pd.to_timedelta(flights_df["DEPARTURE_DELAY"], unit="m")
flights_df["WHEELS_OFF"] = flights_df["DEPARTURE_TIME"] + pd.to_timedelta(flights_df["TAXI_OUT"], unit="m")
flights_df["WHEELS_ON"] = flights_df["WHEELS_OFF"] + pd.to_timedelta(flights_df["AIR_TIME"], unit="m")
flights_df["ARRIVAL_TIME"] = flights_df["WHEELS_ON"] + pd.to_timedelta(flights_df["TAXI_IN"], unit="m")

In [17]:
# flights_df includes flights that are cancelled, diverted and ones that landed at the target airport.  To predict flight delays, I will only be using flights that have landed at the target airports.
landed_df = flights_df.loc[(flights_df["CANCELLED"] == 0) & (flights_df["DIVERTED"] == 0)]

In [18]:
# Cleaning landed_df
landed_df.loc[:,["AIR_SYSTEM_DELAY", "SECURITY_DELAY", "AIRLINE_DELAY", "LATE_AIRCRAFT_DELAY", "WEATHER_DELAY"]] = landed_df.loc[:,["AIR_SYSTEM_DELAY", "SECURITY_DELAY", "AIRLINE_DELAY", "LATE_AIRCRAFT_DELAY", "WEATHER_DELAY"]].fillna(0.0)
landed_df = landed_df.drop(["YEAR", "DIVERTED", "CANCELLED", "CANCELLATION_REASON"], axis=1)

In [19]:
landed_df.dtypes

MONTH                                                        uint8
DAY                                                          uint8
DAY_OF_WEEK                                                  uint8
AIRLINE                                                   category
FLIGHT_NUMBER                                             category
TAIL_NUMBER                                               category
ORIGIN_AIRPORT                                              object
DESTINATION_AIRPORT                                         object
SCHEDULED_DEPARTURE                                 datetime64[ns]
DEPARTURE_TIME                                      datetime64[ns]
DEPARTURE_DELAY                                            float64
TAXI_OUT                                                   float64
WHEELS_OFF                                          datetime64[ns]
SCHEDULED_TIME                                             float64
ELAPSED_TIME                                               flo

In [25]:
landed_df = landed_df.astype({
    "ORIGIN_AIRPORT":"category",
    "DESTINATION_AIRPORT":"category",
    "DEPARTURE_DELAY":"int16",
    "TAXI_OUT":"uint16",
    "SCHEDULED_TIME":"uint16",
    "ELAPSED_TIME":"uint16",
    "AIR_TIME":"uint16",
    "DISTANCE":"uint16",
    "TAXI_IN":"uint8",
    "ARRIVAL_DELAY":"int16",
    "AIR_SYSTEM_DELAY":"uint16",
    "SECURITY_DELAY":"uint16",
    "AIRLINE_DELAY":"uint16",
    "LATE_AIRCRAFT_DELAY":"uint16",
    "WEATHER_DELAY":"uint16"
})

In [26]:
landed_df.to_pickle("landed_flights.pkl")